In [11]:
from ta import *
from sklearn import svm
from sklearn.metrics import confusion_matrix
import pandas as pd
import os
file_list=os.listdir("./Data")


total_data=pd.read_csv("Data/"+file_list[0])
    
stock_index=file_list[0][0:4]

column_change={"調整後終値":"price_"+stock_index,
               "出来高":"volume_"+stock_index, 
               "日付":"date",
                "高値":"high_"+stock_index, 
               "安値":"low_"+stock_index}
total_data=total_data.rename(columns=column_change)
total_data=total_data[["price_"+stock_index,
               "volume_"+stock_index,
               "high_"+stock_index, 
               "low_"+stock_index,
               "date"]]
del file_list[0]
 
for file in file_list:
    stock_index=file[0:4]
    column_change={"調整後終値":"price_"+stock_index,
               "出来高":"volume_"+stock_index, 
               "日付":"date",
                "高値":"high_"+stock_index, 
               "安値":"low_"+stock_index}
    data=pd.read_csv("Data/"+file)
    data=data.rename(columns=column_change)
    data=data[["price_"+stock_index,
               "volume_"+stock_index,
               "high_"+stock_index, 
               "low_"+stock_index,
               "date"]]
    
    total_data=pd.merge(total_data, data, on="date", how="inner")



In [12]:
from datetime import datetime 
total_data["date"]=total_data["date"].apply(lambda x:datetime.strptime(x,"%Y/%m/%d"))
total_data=total_data.sort_values("date", ascending=True)
total_data=total_data.reset_index(drop=True)

In [13]:
collist=total_data.columns
collist_price = list(filter(lambda s : s.find("price")>-1, collist))
collist_high = list(filter(lambda s : s.find("high")>-1, collist))
collist_low = list(filter(lambda s : s.find("low")>-1, collist))
collist_vol = list(filter(lambda s : s.find("volume")>-1, collist))

In [14]:
for i in range(0,len(collist_price)):
    total_data[collist_price[i]+ "_AO"]=momentum.ao(total_data[collist_high[i]],total_data[collist_low[i]])

    total_data[collist_price[i]+ "_TSI"]=momentum.tsi(total_data[collist_price[i]])
    total_data[collist_price[i]+ "_UO"]=momentum.uo(total_data[collist_high[i]],total_data[collist_low[i]],
                                                     total_data[collist_price[i]])
    total_data[collist_price[i]+ "_STOCH_SIGN"]=momentum.stoch_signal(total_data[collist_high[i]],total_data[collist_low[i]],
                                                     total_data[collist_price[i]])
    total_data[collist_price[i]+ "_RSI10"]=momentum.rsi(total_data[collist_price[i]],n=10)
    total_data[collist_price[i]+ "_RSI6"]=momentum.rsi(total_data[collist_price[i]],n=6)
    total_data[collist_price[i]+ "_MFI"]=momentum.money_flow_index(total_data[collist_high[i]],total_data[collist_low[i]],
                                                                  total_data[collist_price[i]],total_data[collist_vol[i]])
   
    total_data[collist_price[i]+ "_wr6"]=momentum.wr(total_data[collist_high[i]],total_data[collist_low[i]],
                                                    total_data[collist_price[i]], lbp=6)
    #std =total_data[collist_price[i]+ "_wr6"].std()
    #mean=total_data[collist_price[i]+ "_wr6"].mean()
    #total_data[collist_price[i]+ "_wr6_norm"]=total_data[collist_price[i]+ "_wr6"].apply(lambda x:(x-mean/std if pd.notnull(x) else 0))
    
    total_data[collist_price[i]+ "_wr10"]=momentum.wr(total_data[collist_high[i]],total_data[collist_low[i]],
                                                    total_data[collist_price[i]], lbp=10)
    #std =total_data[collist_price[i]+ "_wr10"].std()
    #mean=total_data[collist_price[i]+ "_wr10"].mean()
    #total_data[collist_price[i]+ "_wr10_norm"]=total_data[collist_price[i]+ "_wr10"].apply(lambda x:(x-mean/std if pd.notnull(x) else 0))
    total_data[collist_price[i]+ "_macd"]=trend.macd_signal(total_data[collist_price[i]])
    total_data[collist_price[i]+ "_FI"]=volume.force_index(total_data[collist_price[i]], total_data[collist_vol[i]])
    total_data[collist_price[i]+ "_ATR"]=volatility.average_true_range(total_data[collist_high[i]],total_data[collist_low[i]],
                                                                    total_data[collist_price[i]])
    total_data[collist_price[i]+ "_CCI"]=trend.cci(total_data[collist_high[i]],total_data[collist_low[i]],
                                                                    total_data[collist_price[i]])
    total_data[collist_price[i]+ "_KST_SIGN"]=trend.kst_sig(total_data[collist_price[i]])


In [15]:
for price_col in list(collist_price):
    total_data[price_col + "_diff"] = total_data[price_col] - total_data[price_col].shift(1)    
    total_data[price_col+"_diff_sign"] = total_data[price_col+"_diff"].apply(lambda x : 1 if x>=0 else 0)
    total_data[price_col+"_ratio"] = total_data[price_col]/total_data[price_col].shift(1) - 1

In [16]:
collist_price_index=list(map(lambda s: s[-4:], collist_price))

In [17]:
long_data=pd.DataFrame()
for i in range(0,len(collist_price_index)):
    col_each_stock_data = list(filter(lambda s : s.find(collist_price_index[i])>-1, list(total_data.columns)))
    
    col_each_stock_data.append("date")
    long_data_each = total_data[col_each_stock_data]
    long_data_each = long_data_each.drop(["price_"+collist_price_index[i],
                                          "high_"+collist_price_index[i],
                                          "low_"+collist_price_index[i],
                                          "volume_"+collist_price_index[i]],axis=1)
    long_data_each = long_data_each.rename(columns=lambda s: s[11:] if s!="date" else s)
    
    long_data_each["stock_index"] = int(collist_price_index[i])
 
    long_data = pd.concat([long_data,long_data_each])

long_data["year"]=long_data["date"].apply(lambda x:x.year)

In [18]:
fin_data=pd.read_csv("financial_statement.csv")
est_data=pd.merge(long_data,fin_data, on=["stock_index","year"], how="outer")

est_data=est_data[(est_data["date"]>=datetime.strptime("2018-01-01","%Y-%m-%d"))
                           &(est_data["date"]<=datetime.strptime("2019-10-31","%Y-%m-%d"))]


std =est_data["wr6"].std()
mean=est_data["wr6"].mean()
est_data["wr6_norm"]=est_data["wr6"].apply(lambda x:(x-mean/std if pd.notnull(x) else 0))

std =est_data["wr10"].std()
mean=est_data["wr10"].mean()
est_data["wr10_norm"]=est_data["wr10"].apply(lambda x:(x-mean/std if pd.notnull(x) else 0))

std =est_data["AO"].std()
mean=est_data["AO"].mean()
est_data["AO"]=est_data["AO"].apply(lambda x:(x-mean/std if pd.notnull(x) else 0))

In [19]:
learning_data=est_data[(est_data["date"]>=datetime.strptime("2018-01-01","%Y-%m-%d"))
                           &(est_data["date"]<=datetime.strptime("2019-08-31","%Y-%m-%d"))]
test_data=est_data[(est_data["date"]>=datetime.strptime("2019-09-01","%Y-%m-%d"))
                           &(est_data["date"]<=datetime.strptime("2019-10-31","%Y-%m-%d"))]
test_data=test_data.reset_index(drop=True)

In [31]:
y_train=learning_data["diff_sign"]
y_test=test_data["diff_sign"]

x_columns=[
           'wr6_norm',
           'wr10_norm',
           'RSI10',
           'RSI6',
           'STOCH_SIGN',
  #         'KST_SIGN',
           'TSI',
           'UO',
           'AO',
           'ATR',
           'macd',
           'MFI',
           'CCI',
           'equity-ratio',
           'log_sales',
           'log_profit']

x_train=learning_data[x_columns]
x_test=test_data[x_columns]

In [32]:
from sklearn.svm import SVC
model = SVC(random_state=0,gamma='scale')

model.fit(x_train, y_train)
y_pred = model.predict(x_test)
confusion_matrix(y_test, y_pred)

array([[204,  73],
       [ 52, 351]], dtype=int64)